# Predictive Policing in SF - a toy WMD

## For a blog post explaining the motivation behind this project and an interactive map with the predictions visit:

 http://www.orlandotorres.org/predictive-policing-sf.html

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import geopandas
from shapely.geometry import Point
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

pd.options.mode.chained_assignment = None

ImportError: cannot import name 'display' from 'IPython.core.display' (C:\Users\melka\Documents\venv\Lib\site-packages\IPython\core\display.py)

In [ ]:
crime17 = pd.read_csv('sf_incidents17.csv')
crime16 = pd.read_csv('sf_incidents16.csv')


### Find the zip codes for each incident given the latitude and longitude

In [ ]:
crime16['geometry'] = crime16.apply(lambda row: Point(row['X'], row['Y']), axis=1)

In [ ]:
geo_police_data = geopandas.GeoDataFrame(crime16, geometry='geometry')
geo_police_data.crs = {'init': 'epsg:4326'}

In [ ]:
sf = geopandas.read_file('zipcodes.geojson')
sf.crs = {'init': 'epsg:4326'}
sf = sf.set_geometry('geometry')

In [ ]:
crime16 = geopandas.tools.sjoin(geo_police_data, sf, how='left')

### Find which zip codes, day of week, and hour have the most crimes

In [ ]:
day_time_zip_16 = crime16[['Date', 'DayOfWeek', 'Time', 'zip']]

In [ ]:
day_time_zip_16.loc[:, 'Date'] = pd.to_datetime(day_time_zip_16['Date'])

day_time_zip_16.loc[:, 'Hour'] = pd.to_datetime(day_time_zip_16['Time'])
day_time_zip_16.loc[:, 'Hour'] = day_time_zip_16.Hour.apply(lambda x: x.hour)

In [ ]:
day_time_zip_16.head()

## Remove data from November and December because I don't have that data for 2017 to compare it to

In [ ]:
day_time_zip_16_final = day_time_zip_16[day_time_zip_16.Date.dt.month < 11]

## Combine all the crimes into hours and days and zip codes

In [ ]:
day_time_zip_16_final = day_time_zip_16[['DayOfWeek', 'zip', 'Hour']]

In [ ]:
day_time_zip_16_final.loc[:, 'Crimes'] = 1

In [ ]:
hour_totals_16 = day_time_zip_16_final.groupby(['DayOfWeek', 'zip', 'Hour']).sum().reset_index()

In [ ]:
hour_totals_16.sort_values('Crimes', ascending = False).head()

## Create dummy variables to start doing analysis

In [ ]:
hour_totals_16 = hour_totals_16[['Crimes', 'Hour', 'DayOfWeek', 'zip']]

In [ ]:
totals_dummies_16 = pd.get_dummies(hour_totals_16)

In [ ]:
X_16 = totals_dummies_16.iloc[:, 1:]
y_16 = totals_dummies_16.iloc[:, 0]

## Linear Regression with 2016 data

In [ ]:
linear_regression = sm.OLS(y_16, X_16)
results = linear_regression.fit()

In [ ]:
results.summary()

# Test it with new 2017 data

### Find the zip codes for each incident given the latitude and longitude

In [ ]:
crime17['geometry'] = crime17.apply(lambda row: Point(row['X'], row['Y']), axis=1)

In [ ]:
#geo_police_data = geopandas.GeoDataFrame(crime17, geometry='geometry')
#geo_police_data.crs = {'init': 'epsg:4326'}

In [ ]:
geo_police_data = geopandas.GeoDataFrame(crime17, geometry='geometry')
#geo_police_data.crs = {'init': 'epsg:4326'}
geo_police_data.set_crs("EPSG:4326", inplace=True)

In [ ]:
crime17 = geopandas.tools.sjoin(geo_police_data, sf, how='left')

### Find which zip codes, day of week, and hour have the most crimes

In [ ]:
day_time_zip_17 = crime17[['DayOfWeek', 'Time', 'zip']]

In [ ]:
day_time_zip_17['Hour'] = pd.to_datetime(day_time_zip_17['Time'])
day_time_zip_17['Hour'] = day_time_zip_17.Hour.apply(lambda x: x.hour)

## Combine all the crimes into hours and days and zip codes

In [ ]:
day_time_zip_17 = day_time_zip_17[['DayOfWeek', 'zip', 'Hour']]

In [ ]:
day_time_zip_17['Crimes'] = 1

In [ ]:
hour_totals_17 = day_time_zip_17.groupby(['DayOfWeek', 'zip', 'Hour']).count().reset_index()

In [ ]:
hour_totals_17.sort_values('Crimes', ascending = False).head()

## Create dummy variables to start doing analysis

In [ ]:
hour_totals_17 = hour_totals_17[['Crimes', 'Hour', 'DayOfWeek', 'zip']]

In [ ]:
totals_dummies_17 = pd.get_dummies(hour_totals_17)

In [ ]:
X_17 = totals_dummies_17.iloc[:, 1:]
y_17 = totals_dummies_17.iloc[:, 0]

# Testing the different models

## Linear Regression

In [ ]:
linear_regression = LinearRegression()
linear_regression.fit(X_16,y_16)
linear_regression.score(X_17, y_17)

## Random Forest

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_16, y_16)
rf.score(X_17,y_17)

In [ ]:
list(zip(X_16.columns, rf.feature_importances_))

## KNN

In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_16, y_16)
knn.score(X_17,y_17)

## SVM

In [ ]:
svm = SVR()
svm.fit(X_16, y_16)
svm.score(X_17,y_17)

## XGBoost

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_16, y_16)
xgb.score(X_17,y_17)

## MLP Regressor

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes = (100,100,100,100), random_state=444)
mlp.fit(X_16,y_16)

In [ ]:
mlp.score(X_17, y_17)

## Combine predictions and actual results into one dataframe

In [ ]:
mlp_predicts = mlp.predict(X_16)

In [ ]:
xgb_predicts = xgb.predict(X_16)

In [ ]:
hour_totals_17['Predicted_mlp'] = pd.Series(mlp_predicts)
hour_totals_17['Predicted_xgb'] = pd.Series(xgb_predicts)

### Divide by 365 to get the number of crime incidents each day

In [ ]:
hour_totals_17['Crimes'] = hour_totals_17['Crimes']/365
hour_totals_17['Predicted_mlp'] = hour_totals_17['Predicted_mlp']/365
hour_totals_17['Predicted_xgb'] = hour_totals_17['Predicted_xgb']/365


In [ ]:
hour_totals_17 = np.round(hour_totals_17,2)

In [ ]:
hour_totals_17.to_json("./colombia/crime_predictions.json", orient='records', double_precision=2)

## Create a quick Chloropleth map as a sanity check

In [ ]:
chloropleth_data = hour_totals_17.merge(sf)
chloropleth_data = geopandas.GeoDataFrame(chloropleth_data, geometry='geometry')
chloropleth_data.crs = {'init': 'epsg:4326'}

In [ ]:
chloropleth_data.plot(column='Crimes', cmap='OrRd', figsize=(13,10))

### The next step is to make an interactive visualization. For this, I will move to D3.js